In [23]:
import glob
from pathlib import Path

import fsspec
import geopandas as gpd
import xarray as xr
import intake

import hvplot.pandas  # noqa
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.io.img_tiles as cimgt
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import hvplot.xarray

import echopype as ep

import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

### Utils

- convert_files


In [20]:
def download_data(path='data'):
    pass


def combine_data(path='data'):
    """

    :param path:
    :return:
    """

    paths = glob.glob(f'{path}/*.nc')
    assert len(paths) >= 1, 'No NetCDF files present.'

    temp = []
    for path in paths:
        file = ep.open_converted(path)
        if not isinstance(file.platform, xr.core.dataset.Dataset):
            print(f'Skipping file {path} since it does not have latitude/longitude')
        else:
            temp.append(file)

    return ep.combine_echodata(temp)


def get_geo(df):
    geo = df.platform.latitude.to_dataframe()\
          .join(df.platform.longitude.to_dataframe())
    return gpd.GeoDataFrame(
            geo,
            geometry=gpd.points_from_xy(geo['longitude'],
                                        geo['latitude']),
            crs='epsg:4326')


In [14]:
files = combine_data()
files

Skipping file data\Summer2017-D20170625-T190753_Sv.nc since it does not have latitude/longitude


EchoData: standardized raw data from data\Summer2017-D20170728-T181619.nc
  > top: (Top-level) contains metadata about the SONAR-netCDF4 file format.
  > environment: (Environment) contains information relevant to acoustic propagation through water.
  > platform: (Platform) contains information about the platform on which the sonar is installed.
  > nmea: (Platform/NMEA) contains information specific to the NMEA protocol.
  > provenance: (Provenance) contains metadata about how the SONAR-netCDF4 version of the data were obtained.
  > sonar: (Sonar) contains specific metadata for the sonar system.
  > beam: (Beam) contains backscatter data and other beam or channel-specific data.
  > vendor: (Vendor specific) contains vendor-specific information about the sonar and the data.

In [21]:
geo_df = get_geo(files)
geo_df

,latitude,longitude,geometry
location_time,,,
2017-07-28 18:16:21.475999744,43.657532,-124.887015,POINT (-124.88702 43.65753)
2017-07-28 18:16:21.635000320,43.657500,-124.887000,POINT (-124.88700 43.65750)
2017-07-28 18:16:22.168999936,43.657532,-124.887080,POINT (-124.88708 43.65753)
2017-07-28 18:16:23.263000064,43.657532,-124.887147,POINT (-124.88715 43.65753)
2017-07-28 18:16:23.421000192,43.657500,-124.887167,POINT (-124.88717 43.65750)
...,...,...,...
2017-07-28 20:26:07.448999936,43.775167,-125.154667,POINT (-125.15467 43.77517)
2017-07-28 20:26:08.287000064,43.775265,-125.154647,POINT (-125.15465 43.77526)
2017-07-28 20:26:09.035999744,43.775310,-125.154635,POINT (-125.15463 43.77531)


In [24]:
geo_df.hvplot(geo=True, tiles=True)#%%

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Longitude,Latitude]